In [37]:
import pandas as pd
import numpy as np

### Fetching All Excel Files Data

In [38]:
X_Company_Orders = pd.read_excel("R:\Projects\Python Project\cointab assignment\Company X - Order Report.xlsx", sheet_name = "Sheet1")
print(X_Company_Orders.shape)
X_Company_Orders.head()

(400, 3)


,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1
1,2001827036,8904223819093,1
2,2001827036,8904223819109,1
3,2001827036,8904223818430,1
4,2001827036,8904223819277,1


In [39]:
X_Company_SKU_Master = pd.read_excel("R:\Projects\Python Project\cointab assignment\Company X - SKU Master.xlsx",sheet_name = "Sheet1")
print(X_Company_SKU_Master.shape)
X_Company_SKU_Master.head()

(66, 2)


,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [40]:
X_Company_Pincodes = pd.read_excel("R:\Projects\Python Project\cointab assignment\Company X - Pincode Zones.xlsx",sheet_name ="Sheet1")
print(X_Company_Pincodes.shape)
X_Company_Pincodes.head()

(124, 3)


,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [41]:
Courier_Company_Invoice = pd.read_excel("R:\Projects\Python Project\cointab assignment\Courier Company - Invoice.xlsx", sheet_name = "Sheet1")
Courier_Company_Invoice['Zone'] = Courier_Company_Invoice['Zone'].str.upper()
print(Courier_Company_Invoice.shape)
Courier_Company_Invoice.head()

(124, 8)


,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1,121003,507101,D,Forward charges,135
1,1091117222194,2001806273,1,121003,486886,D,Forward charges,90
2,1091117222931,2001806408,2,121003,532484,D,Forward charges,225
3,1091117223244,2001806458,1,121003,143001,B,Forward charges,61
4,1091117229345,2001807012,0,121003,515591,D,Forward charges,45


In [42]:
Courier_Company_Rates = pd.read_excel("R:\Projects\Python Project\cointab assignment\Courier Company - Rates.xlsx", sheet_name= "Sheet2")
print(Courier_Company_Rates.shape)
Courier_Company_Rates

(5, 6)


,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0,30,24,14,24
1,B,1,33,28,20,28
2,C,1,40,39,32,39
3,D,2,45,45,41,45
4,E,2,57,56,51,56


### Caluculating Total Weight For X Company's Each Order Number in Kgs 

1. Joining X_Company_Orders and X_Company_SKU_Master on "SKU" to get respective Weights for order numbers.
2. Calculating total weight in garms based on the ordered quantity.

In [43]:
Weights_df = X_Company_Orders.merge(X_Company_SKU_Master, on ="SKU")
Weights_df['Total Weight(g)'] = (Weights_df['Order Qty']*Weights_df['Weight (g)'])
Weights_df.head()

,ExternOrderNo,SKU,Order Qty,Weight (g),Total Weight(g)
0,2001827036,8904223818706,1,127,127
1,2001821995,8904223818706,1,127,127
2,2001819252,8904223818706,1,127,127
3,2001816996,8904223818706,1,127,127
4,2001814580,8904223818706,1,127,127


1. As there are multiple orders with same order number, grouping order numbers and adding the weight.
2. Converting weights from grams(g) into kilograms(Kgs) and rounding it to 3 decimals
3. Fetching only required columns ExternOrderNo	and Total Weight(Kgs)

In [44]:
X_Company_Total_Weights = Weights_df.groupby('ExternOrderNo')['Total Weight(g)'].sum().reset_index(name="Total Weight(g)")
X_Company_Total_Weights['Total Weight(Kgs)'] = round((X_Company_Total_Weights['Total Weight(g)']/1000),2)
X_Company_Total_Weights = X_Company_Total_Weights[['ExternOrderNo','Total Weight(Kgs)']]
X_Company_Total_Weights.head()

,ExternOrderNo,Total Weight(Kgs)
0,2001806210,0
1,2001806226,0
2,2001806229,0
3,2001806232,1
4,2001806233,0


### Merging X_Company_Pincodes, Courier_Company_Invoice, and X_Company_Total_Weights Data Frames to Create X_Company_Invoice

1. Merging X_Company_Pincodes and Courier_Company_Invoice on 'Warehouse Pincode' and 'Customer Pincode' columns to fetch 'Type of Shipment' and 'Zone' associated with order id.
2. Fetching only required columns from the X_Company_Details data frame.
3. Droping duplicates rows.

In [45]:
X_Company_Details  = X_Company_Pincodes.merge(Courier_Company_Invoice, on = ['Warehouse Pincode','Customer Pincode'], suffixes = ['_X','_C'])
X_Company_Details  = X_Company_Details[['Order ID','AWB Code','Type of Shipment','Zone_X','Warehouse Pincode','Customer Pincode']]
X_Company_Details.drop_duplicates(inplace =True)
print(X_Company_Details.shape)
X_Company_Details.head()

(124, 6)


,Order ID,AWB Code,Type of Shipment,Zone_X,Warehouse Pincode,Customer Pincode
0,2001806232,1091117222124,Forward charges,d,121003,507101
1,2001806273,1091117222194,Forward charges,d,121003,486886
2,2001806408,1091117222931,Forward charges,d,121003,532484
3,2001806458,1091117223244,Forward charges,b,121003,143001
4,2001807012,1091117229345,Forward charges,d,121003,515591


1. Merging X_Company_Details and X_Company_Total_Weights to fetch together ORDER ID's Type of Shipment,Zone, and Total Weight(Kgs).
2. Converting Zone letters to upper case.
3. Fetching all the required columns.

In [46]:
X_Company_Invoice = X_Company_Details.merge(X_Company_Total_Weights, left_on = "Order ID", right_on = "ExternOrderNo")
X_Company_Invoice["Zone"] = X_Company_Invoice["Zone_X"].str.upper()
X_Company_Invoice = X_Company_Invoice[['Order ID','AWB Code','Type of Shipment','Zone','Total Weight(Kgs)','Warehouse Pincode','Customer Pincode']]
print(X_Company_Invoice.shape)
X_Company_Invoice.head()

(124, 7)


,Order ID,AWB Code,Type of Shipment,Zone,Total Weight(Kgs),Warehouse Pincode,Customer Pincode
0,2001806232,1091117222124,Forward charges,D,1,121003,507101
1,2001806273,1091117222194,Forward charges,D,1,121003,486886
2,2001806408,1091117222931,Forward charges,D,2,121003,532484
3,2001806458,1091117223244,Forward charges,B,1,121003,143001
4,2001807012,1091117229345,Forward charges,D,0,121003,515591


1. Merging X_Company_Invoice and X_Company_Rates to Fetch Slab Length and Prices For Orders
2. Renaming Weight Slabs column to Slab Length 

In [47]:
X_Company_Invoice_df = X_Company_Invoice.merge(Courier_Company_Rates,on= "Zone")
X_Company_Invoice_df = X_Company_Invoice_df.rename(columns = {'Weight Slabs' : 'Slab Length'})
X_Company_Invoice_df.head()

,Order ID,AWB Code,Type of Shipment,Zone,Total Weight(Kgs),Warehouse Pincode,Customer Pincode,Slab Length,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,2001806232,1091117222124,Forward charges,D,1,121003,507101,2,45,45,41,45
1,2001806273,1091117222194,Forward charges,D,1,121003,486886,2,45,45,41,45
2,2001806408,1091117222931,Forward charges,D,2,121003,532484,2,45,45,41,45
3,2001807012,1091117229345,Forward charges,D,0,121003,515591,2,45,45,41,45
4,2001806686,1091117229555,Forward charges,D,0,121003,326502,2,45,45,41,45


1. For Forward charges type of shipment RTO Fixed Charge and RTO Additional Weight Slab Charge are set to null.

In [48]:
X_Company_Invoice_df.loc[X_Company_Invoice_df['Type of Shipment'] == "Forward charges", ['RTO Fixed Charge','RTO Additional Weight Slab Charge']]  =np.nan,np.nan
print(X_Company_Invoice_df.shape)
X_Company_Invoice_df.head()

(124, 12)


,Order ID,AWB Code,Type of Shipment,Zone,Total Weight(Kgs),Warehouse Pincode,Customer Pincode,Slab Length,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,2001806232,1091117222124,Forward charges,D,1,121003,507101,2,45,45,NaN,NaN
1,2001806273,1091117222194,Forward charges,D,1,121003,486886,2,45,45,NaN,NaN
2,2001806408,1091117222931,Forward charges,D,2,121003,532484,2,45,45,NaN,NaN
3,2001807012,1091117229345,Forward charges,D,0,121003,515591,2,45,45,NaN,NaN
4,2001806686,1091117229555,Forward charges,D,0,121003,326502,2,45,45,NaN,NaN


1. Calculating Weight Slab by using Slab Length and Total weight(Kgs).
2. Formula:-  Weight Slab = np.ceil(Total Weight/Slab Length)*Slab Length 
3. Appending weight_slab into a list, such that each index weight_slab is in respective order.
4. Adding Weight Slab column to X_Company_Invoice_df

In [49]:
weight_slab_list = []
for i in X_Company_Invoice_df.index:
    row = X_Company_Invoice_df.iloc[i]
    weight_slab = np.ceil(row['Total Weight(Kgs)'] / row['Slab Length']) * row['Slab Length']
    weight_slab_list.append(weight_slab)
    

X_Company_Invoice_df['Weight_Slab'] = weight_slab_list
X_Company_Invoice_df.sort_index(ascending = True).head()

,Order ID,AWB Code,Type of Shipment,Zone,Total Weight(Kgs),Warehouse Pincode,Customer Pincode,Slab Length,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Weight_Slab
0,2001806232,1091117222124,Forward charges,D,1,121003,507101,2,45,45,NaN,NaN,2
1,2001806273,1091117222194,Forward charges,D,1,121003,486886,2,45,45,NaN,NaN,2
2,2001806408,1091117222931,Forward charges,D,2,121003,532484,2,45,45,NaN,NaN,3
3,2001807012,1091117229345,Forward charges,D,0,121003,515591,2,45,45,NaN,NaN,2
4,2001806686,1091117229555,Forward charges,D,0,121003,326502,2,45,45,NaN,NaN,2


### Calculating Total Price For Each Order ID In X_Company_Invoice

1. For "Forward charges" type of shipment,Forward Fixed Charge and Forward Additional Weight Slab Charge are considered.
2. For "Forward and RTO charges" type of shipment, Forward Fixed Charge, Forward Additional Weight Slab Charge, RTO Fixed Charge, and RTO Additional Weight Slab Charge are considered.
3. Formula to calculate total price =  fixed_price + additioncal_price
4. For 1st slab length in weight slab fixed charge is considered. For all extar slabs additional charge is considered.
5. additional price = additional charge*(weight slab/ slab length -1)  

In [50]:

total_charge_list = []
for i in X_Company_Invoice_df.index:
    row = X_Company_Invoice_df.iloc[i]
    if row['Type of Shipment'] == "Forward charges":
        fixed_price = row['Forward Fixed Charge']
        additional_price = row['Forward Additional Weight Slab Charge']* (row['Weight_Slab']/row['Slab Length'] - 1)
        total_price = fixed_price + additional_price
        total_charge_list.append(total_price)
    elif row['Type of Shipment'] == "Forward and RTO charges":
        fixed_price = row['Forward Fixed Charge'] + row['RTO Fixed Charge']
        additional_price = (row['RTO Additional Weight Slab Charge']+row['Forward Additional Weight Slab Charge'])* (row['Weight_Slab']/row['Slab Length'] - 1)
        total_price = fixed_price + additional_price
        total_charge_list.append(total_price)
  


X_Company_Invoice_df['Expected Charge'] = total_charge_list
X_Company_Invoice_df

,Order ID,AWB Code,Type of Shipment,Zone,Total Weight(Kgs),Warehouse Pincode,Customer Pincode,Slab Length,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Weight_Slab,Expected Charge
0,2001806232,1091117222124,Forward charges,D,1,121003,507101,2,45,45,NaN,NaN,2,45
1,2001806273,1091117222194,Forward charges,D,1,121003,486886,2,45,45,NaN,NaN,2,45
2,2001806408,1091117222931,Forward charges,D,2,121003,532484,2,45,45,NaN,NaN,3,90
3,2001807012,1091117229345,Forward charges,D,0,121003,515591,2,45,45,NaN,NaN,2,45
4,2001806686,1091117229555,Forward charges,D,0,121003,326502,2,45,45,NaN,NaN,2,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,Forward charges,E,1,121003,173212,2,57,56,NaN,NaN,2,57
120,2001811475,1091121844806,Forward charges,E,1,121003,173212,2,57,56,NaN,NaN,2,57
121,2001819252,1091120352712,Forward charges,E,0,121003,174101,2,57,56,NaN,NaN,2,57
122,2001827036,1091122418320,Forward charges,E,2,121003,173213,2,57,56,NaN,NaN,4,112


1. Fetching only required columns AWB Code, Order ID, Total Weight(Kgs), Zone, Type of Shipment, Weight_Slab, and Expected Charge.

In [51]:
X_Company_Invoice_df = X_Company_Invoice_df[['Order ID','AWB Code','Total Weight(Kgs)','Warehouse Pincode','Customer Pincode','Zone','Type of Shipment','Weight_Slab','Expected Charge']]
print(X_Company_Invoice_df.shape)
X_Company_Invoice_df

(124, 9)


,Order ID,AWB Code,Total Weight(Kgs),Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Weight_Slab,Expected Charge
0,2001806232,1091117222124,1,121003,507101,D,Forward charges,2,45
1,2001806273,1091117222194,1,121003,486886,D,Forward charges,2,45
2,2001806408,1091117222931,2,121003,532484,D,Forward charges,3,90
3,2001807012,1091117229345,0,121003,515591,D,Forward charges,2,45
4,2001806686,1091117229555,0,121003,326502,D,Forward charges,2,45
...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,1,121003,173212,E,Forward charges,2,57
120,2001811475,1091121844806,1,121003,173212,E,Forward charges,2,57
121,2001819252,1091120352712,0,121003,174101,E,Forward charges,2,57
122,2001827036,1091122418320,2,121003,173213,E,Forward charges,4,112


### Calculating Weight Slab For Courier_Company_Invoice Order Details

1. Merging Courier_Company_Invoice and Courier_Company_Rates to fetch the Slab Length for each order.
2. Fetching only required columns and renaming Weight Slabs column ti Slab Length

In [52]:
Courier_Company_Invoice_df = Courier_Company_Invoice.merge(Courier_Company_Rates, on="Zone")
Courier_Company_Invoice_df = Courier_Company_Invoice_df[['Order ID','AWB Code','Charged Weight','Warehouse Pincode','Customer Pincode','Zone','Type of Shipment','Billing Amount (Rs.)','Weight Slabs']]
Courier_Company_Invoice_df = Courier_Company_Invoice_df.rename(columns= {'Weight Slabs' : 'Slab Length'}).sort_index(ascending = True)
print(Courier_Company_Invoice_df.shape)
Courier_Company_Invoice_df.head()

(124, 9)


,Order ID,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Slab Length
0,2001806232,1091117222124,1,121003,507101,D,Forward charges,135,2
1,2001806273,1091117222194,1,121003,486886,D,Forward charges,90,2
2,2001806408,1091117222931,2,121003,532484,D,Forward charges,225,2
3,2001807012,1091117229345,0,121003,515591,D,Forward charges,45,2
4,2001806686,1091117229555,0,121003,326502,D,Forward charges,45,2


1. Calculating Weight Slab by using Slab Length and Total weight(Kgs).
2. Formula:-  Weight Slab = np.ceil(Charged Weight/Slab Length)*Slab Length 
3. Appending weight_slab into a list, such that each index weight_slab is in respective order.
4. Adding Weight Slab column to Courier_Company_Invoice

In [53]:
weight_slab_list = []
for i in Courier_Company_Invoice_df.index:
    row = Courier_Company_Invoice_df.iloc[i]
    weight_slab = np.ceil(row['Charged Weight']/row['Slab Length']) * row['Slab Length']
    weight_slab_list.append(weight_slab)
    

Courier_Company_Invoice_df['Weight Slab'] = weight_slab_list
Courier_Company_Invoice_df.drop(columns = "Slab Length", inplace = True)
Courier_Company_Invoice_df

,Order ID,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Weight Slab
0,2001806232,1091117222124,1,121003,507101,D,Forward charges,135,2
1,2001806273,1091117222194,1,121003,486886,D,Forward charges,90,2
2,2001806408,1091117222931,2,121003,532484,D,Forward charges,225,3
3,2001807012,1091117229345,0,121003,515591,D,Forward charges,45,2
4,2001806686,1091117229555,0,121003,326502,D,Forward charges,45,2
...,...,...,...,...,...,...,...,...,...
119,2001827036,1091122418320,2,121003,173213,B,Forward charges,118,2
120,2001808585,1091117436652,1,121003,175101,B,Forward charges,61,1
121,2001811475,1091121844806,0,121003,173212,B,Forward charges,33,1
122,2001808295,1091117435661,0,121003,673002,E,Forward and RTO charges,107,2


### Merging and Comparing X_Company_Invoice_df and Courier_Company_Invoice_df For Order Level Calculation

1. Merging X_Company_Invoice_df and Courier_Company_Invoice_df on Order ID and AWD Code.
2. Adding suffixes to columns as per respective data frames.

In [54]:
Order_Level_Calculation_df = X_Company_Invoice_df.merge(Courier_Company_Invoice_df, on= ['Order ID','AWB Code'], suffixes = (' as Per X', ' as Per Courier Company'))
print(Order_Level_Calculation_df.shape)
Order_Level_Calculation_df

(124, 16)


,Order ID,AWB Code,Total Weight(Kgs),Warehouse Pincode as Per X,Customer Pincode as Per X,Zone as Per X,Type of Shipment as Per X,Weight_Slab,Expected Charge,Charged Weight,Warehouse Pincode as Per Courier Company,Customer Pincode as Per Courier Company,Zone as Per Courier Company,Type of Shipment as Per Courier Company,Billing Amount (Rs.),Weight Slab
0,2001806232,1091117222124,1,121003,507101,D,Forward charges,2,45,1,121003,507101,D,Forward charges,135,2
1,2001806273,1091117222194,1,121003,486886,D,Forward charges,2,45,1,121003,486886,D,Forward charges,90,2
2,2001806408,1091117222931,2,121003,532484,D,Forward charges,3,90,2,121003,532484,D,Forward charges,225,3
3,2001807012,1091117229345,0,121003,515591,D,Forward charges,2,45,0,121003,515591,D,Forward charges,45,2
4,2001806686,1091117229555,0,121003,326502,D,Forward charges,2,45,0,121003,326502,D,Forward charges,45,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,1,121003,173212,E,Forward charges,2,57,1,121003,173212,B,Forward charges,61,1
120,2001811475,1091121844806,1,121003,173212,E,Forward charges,2,57,0,121003,173212,B,Forward charges,33,1
121,2001819252,1091120352712,0,121003,174101,E,Forward charges,2,57,0,121003,174101,B,Forward charges,33,1
122,2001827036,1091122418320,2,121003,173213,E,Forward charges,4,112,2,121003,173213,B,Forward charges,118,2


1. Removing unwanted columns from the Order_Level_Calculation_df
2. Renaming columns with the required names
3. Reordering columns in the data frame

In [55]:
Order_Level_Calculation_df.drop(columns=['Warehouse Pincode as Per X','Customer Pincode as Per X','Type of Shipment as Per X','Warehouse Pincode as Per Courier Company','Customer Pincode as Per Courier Company','Type of Shipment as Per Courier Company'], inplace = True)
Order_Level_Calculation_df.rename(columns = {'Total Weight(Kgs)':'Total Weight(KG) as Per X'},inplace = True)
Order_Level_Calculation_df.rename(columns = {'Weight_Slab':'Weight Slab as Per X(KG)'},inplace = True)
Order_Level_Calculation_df.rename(columns = {'Expected Charge':'Expected Charge as per X (Rs.)'},inplace = True)
Order_Level_Calculation_df.rename(columns = {'Zone as Per X':'Delivery Zone as per X'},inplace = True)
Order_Level_Calculation_df.rename(columns = {'Zone as Per Courier Company':'Delivery Zone charged by Courier Company'},inplace = True)
Order_Level_Calculation_df.rename(columns = {'Charged Weight':'Total weight as per Courier Company (KG)'},inplace = True)
Order_Level_Calculation_df.rename(columns = {'Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)'},inplace = True)
Order_Level_Calculation_df.rename(columns = {'Weight Slab':'Weight slab charged by Courier Company (KG)'},inplace = True)
Order_Level_Calculation_df = Order_Level_Calculation_df[['Order ID','AWB Code','Total Weight(KG) as Per X','Weight Slab as Per X(KG)','Total weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)','Delivery Zone as per X','Delivery Zone charged by Courier Company','Expected Charge as per X (Rs.)','Charges Billed by Courier Company (Rs.)']]
print(Order_Level_Calculation_df.shape)
Order_Level_Calculation_df


(124, 10)


,Order ID,AWB Code,Total Weight(KG) as Per X,Weight Slab as Per X(KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.)
0,2001806232,1091117222124,1,2,1,2,D,D,45,135
1,2001806273,1091117222194,1,2,1,2,D,D,45,90
2,2001806408,1091117222931,2,3,2,3,D,D,90,225
3,2001807012,1091117229345,0,2,0,2,D,D,45,45
4,2001806686,1091117229555,0,2,0,2,D,D,45,45
...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,1,2,1,1,E,B,57,61
120,2001811475,1091121844806,1,2,0,1,E,B,57,33
121,2001819252,1091120352712,0,2,0,1,E,B,57,33
122,2001827036,1091122418320,2,4,2,2,E,B,112,118


1. Creating new column in Order_Level_Calculation_df 'Difference Between Expected Charges and Billed Charges (Rs.)'
2. Converting Order ID and AWB Code columns to type string, it helps in preventing number from converting to scientific numbers

In [62]:
Order_Level_Calculation_df.loc[:,'Difference Between Expected Charges and Billed Charges (Rs.)'] = round(Order_Level_Calculation_df['Expected Charge as per X (Rs.)'] - Order_Level_Calculation_df['Charges Billed by Courier Company (Rs.)'],2)
Order_Level_Calculation_df.loc[:, 'Order ID'] = Order_Level_Calculation_df['Order ID'].astype(str)
Order_Level_Calculation_df.loc[:, 'AWB Code'] = Order_Level_Calculation_df['AWB Code'].astype(str)
Order_Level_Calculation_df

,Order ID,AWB Code,Total Weight(KG) as Per X,Weight Slab as Per X(KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1,2,1,2,D,D,45,135,-90
1,2001806273,1091117222194,1,2,1,2,D,D,45,90,-45
2,2001806408,1091117222931,2,3,2,3,D,D,90,225,-134
3,2001807012,1091117229345,0,2,0,2,D,D,45,45,0
4,2001806686,1091117229555,0,2,0,2,D,D,45,45,0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,1,2,1,1,E,B,57,61,-5
120,2001811475,1091121844806,1,2,0,1,E,B,57,33,24
121,2001819252,1091120352712,0,2,0,1,E,B,57,33,24
122,2001827036,1091122418320,2,4,2,2,E,B,112,118,-6


### Creating Summary Table from Order_Level_Calculation_df

1. Fetching only required columns from Order_Level_Calculation_df to create Summary Table and renaming columns

In [61]:

Summary_table = Order_Level_Calculation_df[['Order ID','Charges Billed by Courier Company (Rs.)','Difference Between Expected Charges and Billed Charges (Rs.)']].copy()
Summary_table.rename(columns={'Difference Between Expected Charges and Billed Charges (Rs.)':'Charge Difference'}, inplace = True)
Summary_table.rename(columns={'Charges Billed by Courier Company (Rs.)':'Charges Billed'}, inplace = True)
Summary_table.sort_index(ascending = True, inplace = True)
Summary_table

,Order ID,Charges Billed,Charge Difference
0,2001806232,135,-90
1,2001806273,90,-45
2,2001806408,225,-134
3,2001807012,45,0
4,2001806686,45,0
...,...,...,...
119,2001811604,61,-5
120,2001811475,33,24
121,2001819252,33,24
122,2001827036,118,-6


1. Creating a new column in Summary_table data frame to determine if amount is over charged, correctly charged or under charged

In [58]:
status_list = []
for i in Summary_table.index:
    row = Summary_table.iloc[i]
    if row['Charge Difference'] < 0:
        status = "Over Charged"
        status_list.append(status)
    elif row['Charge Difference'] > 0:
        status = "Under Charged"
        status_list.append(status)
    elif row['Charge Difference'] == 0:
        status = "Correctly Charged"
        status_list.append(status)
        
Summary_table.loc[:,'Status'] = status_list
Summary_table

,Order ID,Charges Billed,Charge Difference,Status
0,2001806232,135,-90,Over Charged
1,2001806273,90,-45,Over Charged
2,2001806408,225,-134,Over Charged
3,2001807012,45,0,Correctly Charged
4,2001806686,45,0,Correctly Charged
...,...,...,...,...
119,2001811604,61,-5,Over Charged
120,2001811475,33,24,Under Charged
121,2001819252,33,24,Under Charged
122,2001827036,118,-6,Over Charged


1. Grouping Summary_Table_df  by status to get count and total amount associated with respective status
2. Calculating total amount for correctly charged status and adding it to the Summary_Table_df

In [59]:
Summary_Table_df = Summary_table.groupby(['Status'])['Charge Difference'].agg(['count','sum']).reset_index()
correctly_charged_amount = Summary_table[Summary_table['Status'] == "Correctly Charged"].groupby('Status')['Charges Billed'].sum().reset_index(name = 'Amount')
Summary_Table_df.loc[Summary_Table_df['Status'] == "Correctly Charged",'sum'] = correctly_charged_amount['Amount']
Summary_Table_df.rename(columns = {'sum': 'Amount(Rs.)','count':'Count'}, inplace= True)
Summary_Table_df['Status'] = "Total orders where X has been " + Summary_Table_df['Status']
Summary_Table_df
                            

,Status,Count,Amount(Rs.)
0,Total orders where X has been Correctly Charged,8,429
1,Total orders where X has been Over Charged,114,-7696
2,Total orders where X has been Under Charged,2,47


### Writing Order_Level_Calculation_df and Summary_Table_df To Excel Workbook

In [60]:
with pd.ExcelWriter("Result.xlsx") as w:
    Order_Level_Calculation_df.to_excel(w,sheet_name = "Order Level Calculation", index= False)
    Summary_Table_df.to_excel(w, sheet_name = "Summary Table", index = False)
    
    